# 2025-04-02 GMRES

## Last time

* Cost of sparse solvers on large systems
* Classical iterative methods: gradient descent
* Convergence of gradient descent
* Krylov subspaces

## Today

* How GMRES works (more math than usual, but worth it)

## Resources

* Trefethen & Bau: Numerical Linear Algebra, Ch 33, 35.
* [The GMRES paper](https://epubs.siam.org/doi/10.1137/0907058)
* (extra) [On the convergence rate of GMRES](https://citeseerx.ist.psu.edu/document?repid=rep1&amp;type=pdf&amp;doi=35ee3114bf834b93101a6f02427a021e22204f5a)

## Methods


|               | $Ax = b$                 | <div style="width:290px"> $Ax = \lambda x$</div>         |
|---------------| :------------------------: | :----------------: |
|$ A = A^* $    | CG (conjugate gradient)  | Lánczos          |
|$ A \neq A^* $ | GMRES, CGN, BCG          | Arnoldi          |

* typically $\mathcal{O}(N)$ steps, $\mathcal{O}(N^2)$ work in each in _worst case_
* if successful, can reduce one or both of these factors
* number of iterations depends on spectral properties of $A$ (related to polynomial approximation)
* work per step depends on whether structure of $A$ can be exploited in computing $x \mapsto Ax$
* _best case_: $\mathcal{O}(N) \to \mathcal{O}(1)$ steps, $\mathcal{O}(N^2) \to \mathcal{O}(N)$ work per step.


## Krylov subspaces

* iterative methods project an $N$-dimensional problem down to a lower dimensional **Krylov subspace**
* given vector $b$ and matrix $A$, the set of vectors 
$$b, Ab, A^2b, \ldots $$
computed by a black box (which is only allowed to do matrix-vector multiplication) as 
$$b, Ab, A(Ab), A(A(Ab)), \ldots $$
define subspaces
$$ \text{span}\{ b\}, \text{span}\{b, Ab\}, \text{span}\{ b, Ab, A^2b\}, \ldots $$
* In each method, projection onto Krylov subspaces reduces the original prolem to a sequence of dimensions $n = 1, 2, 3, \ldots$
* Krylov "matrix" $K := [b|Ab|Ab^2|\dots]$ horribly ill-conditioned. **Question**: why?

## Question: Why is the Krylov matrix so ill-conditioned?

* Suppose $A$ has spectrum $\{\lambda_i\}$ and eigenvectors $\{v_i\}$
* Assume an ordering $\lambda_1 \gg \lambda_2 \geq \lambda_3 \geq \lambda4 \ldots$ and scaling such that $\lambda_1 = 1$.
* Then what happens to a vector $u$ not orthogonal to $v_1$ by the repeated application of $A$? 

_Hint_: decompose in terms of eigenvectors of $A$.
* $\rightarrow$ we'll need to construct a better-conditioned basis for the Krylov subspaces (Arnoldi iteration)

## Arnoldi iteration

* Similar to Gram--Schmidt process, for transforming a matrix into [Hessenberg form](https://en.wikipedia.org/wiki/Hessenberg_matrix). That form is "almost triangular" (nonzeros allowed on first sub-/superdiagonal).
* We'll rename $N \to m$ (matrix size) and introduce another positive integer $n < m$. Assume $m$ is huge.
* Complete reduction to Hessenberg form: 
$$ A = QHQ^{\dagger} \quad \text{with $Q$ unitary, i.e. }\; Q^{\dagger}Q = I  $$ 
equivalently
$$ AQ = QH $$
* But since $m$ is huge, we'll only compute the first $n$ columns of $Q$ ($m \times n$ matrix), which we call $Q_n$.
* Let $\tilde{H}_n$ be an $(n+1) \times n $ upper-right section of the upper-right Hessenberg matrix $H$. Then
$$ AQ_n = Q_{n+1} \tilde{H}_n $$
* The $n$th column of this equation is 
$$ Aq_n = q_1h_{1n} + q_2h_{2n} + \ldots + q_{n+1}h_{n+1,n} $$
* $\to$ $q_{n+1}$ satisfies an $(n+1)$-term recurrence relation involving itself and previous Krylov basis vectors!

$$ Aq_n = q_1h_{1n} + q_2h_{2n} + \ldots + q_{n+1}h_{n+1,n} $$
* $b$: arbitrary, $q_1 := b / ||b||$
* **for** $n = 1, 2, 3, \ldots$
    * $v = Aq_n$
    * **for** $j = 1$ to $n$:
        * $h_{jn} := q_j^{\dagger} v$
        * $v := v - h_{jn}q_j$
        * (At this point, $ v = h_{n+1, n} q_{n+1}$)
        * $h_{n+1, n} := ||v||$
        * $q_{n+1} := v / h_{n+1, n}$
* Note that we only need to compute $Aq_n$ (column-by-column operation).
* What this does is iteratively removes the contribution of each of the $q_i$ from $Aq_n$ in the above formula, until we're left with the term that contains $q_{n+1}$.

* Why useful? Projects onto Krylov subspace. To see: $Q_n^{\dagger} Q_{n+1}$ is $n \times (n+1)$ identity matrix,
* so $Q_n^{\dagger} Q_{n+1} \tilde{H}_n$ deletes last row of $\tilde{H}_n$ but it's still in Hessenberg form
* from $AQ_n = Q_{n+1}\tilde{H}_n$, substitute this to get $$H_n = Q_n^{\dagger} A Q_n $$
    * This is a representation of the matrix $A$ in the basis $\{q_1, q_2, \ldots, q_n \}$
    * Equivalently, the orthogonal projection of $A$ into $\mathcal{K}_n$ ($n$th Krylov subspace)
* Now consider the linear operator that maps $\mathcal{K}_n \mapsto \mathcal{K}_n$:
    * given $v \in \mathcal{K}_n$, apply $A$ to it, then orthogonally project $Av$ back into the space $\mathcal{K}_n$
    * Since orthogonal projection of $\mathbb{C}^m$ onto $\mathcal{K}_n$ is $Q_nQ_n^{\dagger}$, this operation can be written 
    $$ Q_nQ_n^{\dagger}A $$
    (with respect to a standard basis in $\mathbb{C}^m$)
    and 
    $$ Q_n^{\dagger}(Q_nQ_n^{\dagger}A)Q_n = Q_n^{\dagger}AQ_n $$
    with respsect to the columns of $Q_n$ (i.e. in the basis defined by $Q_n$).

## GMRES

* $A$ is $m \times m$ complex square matrix, $b$ is $m$-length complex vector, $\mathcal{K}_n$ is $n$th Krylov subspace.
* At step $n$, approximate $x_* = A^{-1}b$ by vector $x_n \in \mathcal{K}_n$ that minimizes norm of 
$$ r_n := b - Ax_n $$
* Obivous way: let $K_n$ be $[b | Ab| A^2b | \ldots | A^{n-1}b]$, an $m \times n$ matrix, then
$$ AK_n = [Ab | A^2b| \ldots | A^{n}b] $$
and find complex coefficient vector $c$ such that $|| AK_nc - b ||$ is minimized (this is unstable)
* Instead use Arnoldi iteration to construct sequence of $Q_n$ matrices whose columns span $\mathcal{K}_n$, and $x_n := Q_n y$ instead of $x_n = K_n c$. 
* so find $n$-element vector $y$ s.t. $|| AQ_ny - b || $ is minimal.
* This looks like an $m \times n$ problem, but is actually $(n+1) \times n$, because
$$ AQ_n = Q_{n+1} \tilde{H}_n  $$
$\to$ we minimize $|| Q_{n+1}\tilde{H}_n y - b ||$, where both terms are in the column space of $Q_{n+1}$.
* Multiplication by $Q_{n+1}^{\dagger}$ doesn't change the norm of the quantity we are minimizing, so instead minimize 
$$ || \tilde{H}_n y - Q_{n+1}^{\dagger} b || $$
(but $Q_{n+1}^{\dagger} b$ is just first basis vector in the $Q$ basis, so it's $||b||[1 0 0 \ldots 0]$ by construction)
* **At each step, GMRES solves this and sets $x_n = Q_n y$.**

* $q_1 = b / ||b||$
* **for** $n = 1, 2, \ldots$:
    * < step n of Arnoldi iteration > 
    * Find $y$ to minimize $r_n := || \tilde{H}_n y - b||e_1|| ||$
    * $x_n := Q_n y$
* Why is it guaranteed to work? $r_{n+1}  \leq r_n$ because $$\mathrm{col}( \mathcal{K}_{n+1}) \supset
\mathrm{col}(\mathcal{K}_{n})$$ (the next Krylov subspace encloses the previous one)
    * i.e. we are performing least squares in a larger space.

## Demo time!

* Check out the matlab code [here](https://github.com/cu-numpde/spring25/blob/main/mygmres.m)
* Julia version coming soon!